In [1]:
from set_math import *
from datasets_init import *

sympy imported
pandas imported
numpy imported
scipy imported


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


statsmodels imported
statsmodels.formula imported
statasmodels.stats.api imported
train.csv good
test.csv good
you may now begin


# Preprocessing

### np.nan value

In [2]:
train.shape

(647054, 7)

In [3]:
train.dropna(how='any', inplace=True)
train.shape

(642925, 7)

#### Weekday

In [4]:
# 요일 numerical encoding

wd = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3, 
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6,
}

#### Scancount

In [5]:
def purchase_return(train):
    """
    add new column return
    """
    train.loc[train.ScanCount < 0, "Return"] = 1
    train.loc[train.Return != 1, "Return"] = 0
    train.loc[train.Return == 1, "ScanCount"] = 0

In [6]:
train["Weekday"] = train["Weekday"].apply(lambda x: wd[x])
purchase_return(train)


### Drop `Upc` , `HEALTH AND BEAUTY AIDS`

In [7]:
train.drop("Upc", axis=1, inplace=True)

In [8]:
train = train[train['DepartmentDescription'] != 'HEALTH AND BEAUTY AIDS']

In [9]:
print(train.shape)
print(train.columns)
train.head()

(642923, 7)
Index(['TripType', 'VisitNumber', 'Weekday', 'ScanCount',
       'DepartmentDescription', 'FinelineNumber', 'Return'],
      dtype='object')


,TripType,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,Return
0,999,5,4,0,FINANCIAL SERVICES,1000.0,1.0
1,30,7,4,1,SHOES,8931.0,0.0
2,30,7,4,1,PERSONAL CARE,4504.0,0.0
3,26,8,4,2,PAINT AND ACCESSORIES,3565.0,0.0
4,26,8,4,2,PAINT AND ACCESSORIES,1017.0,0.0


### 학습을 위한 전처리 준비

In [10]:
def count_per_DD(data):
    """
    OneHotEncoding DepartmentDescription 
    then multiply ScanCount.
    """
    dummies = pd.get_dummies(data.DepartmentDescription)
    data_dummies = dummies.apply(lambda x: x * data["ScanCount"])
    data_dummies = data_dummies.replace(-0, 0)
    data[dummies.columns] = data_dummies

In [11]:
count_per_DD(train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [12]:
train_max_groupby = train[["TripType", "VisitNumber", "Weekday", "Return"]] # max로 groupby 하는 피처들
group_max = train_max_groupby.groupby("VisitNumber").agg(np.max).reset_index()

In [13]:
dpts = train.DepartmentDescription.unique()

In [14]:
train_dpts = train.loc[:, "1-HR PHOTO":"WIRELESS"]
train_scvn = train[["VisitNumber", "ScanCount"]]
train_sum_groupby = pd.concat([train_scvn, train_dpts], axis=1)
print(train_sum_groupby.shape)
train_sum_groupby.head()

(642923, 69)


,VisitNumber,ScanCount,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,7,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
group_sum = train_sum_groupby.groupby("VisitNumber").agg(np.sum).reset_index()
print(group_sum.shape)
group_sum.head()

(94247, 69)


,VisitNumber,ScanCount,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,2,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,8,29,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
triptype_df = train[['TripType', 'VisitNumber']].groupby(['VisitNumber']).mean()
triptype_df.reset_index(drop=False, inplace=True)
triptype_df.shape

(94247, 2)

In [17]:
return_df = train[['Return', 'VisitNumber']].groupby(['VisitNumber']).sum()
return_df['Return'] = return_df['Return'].apply(lambda x: 1 if x > 0 else 0)
return_df.reset_index(drop=False, inplace=True)
return_df.shape

(94247, 2)

In [18]:
weekday_df = train[['Weekday', 'VisitNumber']].groupby(['VisitNumber']).mean()
weekday_df.reset_index(drop=False, inplace=True)
weekday_df.shape

(94247, 2)

In [19]:
group_sum['TripType'] = triptype_df['TripType']
group_sum['Return'] = return_df['Return']
group_sum['Weekday'] = weekday_df['Weekday']

In [32]:
from sklearn.preprocessing import StandardScaler

#scaling refund, purchase 
scaler = StandardScaler()
group_sum['ScanCount'] = scaler.fit_transform(group_sum['ScanCount'].values.reshape(-1, 1))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [33]:
train_df = group_sum.copy()

In [34]:
print(train_df.shape)
train_df.head()

(94247, 72)


,VisitNumber,ScanCount,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,TripType,Return,Weekday
0,5,-0.754431,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,999,1,4
1,7,-0.559743,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,30,0,4
2,8,2.068540,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,26,1,4
3,9,-0.462399,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,4
4,10,-0.462399,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,0,4


In [35]:
X_df = train_df.drop(['TripType', 'VisitNumber'], axis=1)
X_df.shape

(94247, 70)

In [36]:
y_df = train_df[['TripType']]
y_df.shape

(94247, 1)

### Train, Test split

In [37]:
from sklearn.model_selection import train_test_split

X, y = X_df, y_df
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Kneighbors Classifiers

In [38]:
from sklearn.neighbors import KNeighborsClassifier

### 이웃 = 3

In [39]:
k_clf3 = KNeighborsClassifier(n_neighbors=3)
k_clf3.fit(X_train, y_train)
k_clf3.predict(X_test)

print("Train score: {:.3f}".format(k_clf3.score(X_train, y_train)))
print("Test score: {:.3f}".format(k_clf3.score(X_test, y_test)))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


Train score: 0.692
Test score: 0.563


### 이웃 = 7

In [ ]:
k_clf7 = KNeighborsClassifier(n_neighbors=7)
k_clf7.fit(X_train, y_train)
k_clf7.predict(X_test)
print("Train score: {:.3f}".format(k_clf7.score(X_train, y_train)))
print("Test score: {:.3f}".format(k_clf7.score(X_test, y_test)))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


Train score: 0.641
Test score: 0.578


### 이웃 = 10

In [ ]:
k_clf10 = KNeighborsClassifier(n_neighbors=10)
k_clf10.fit(X_train, y_train)
k_clf10.predict(X_test)
print("Train score: {:.3f}".format(k_clf10.score(X_train, y_train)))
print("Test score: {:.3f}".format(k_clf10.score(X_test, y_test)))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


### 이웃=13 

In [ ]:
k_clf13 = KNeighborsClassifier(n_neighbors=13)
k_clf13.fit(X_train, y_train)
k_clf13.predict(X_test)
print("Train score: {:.3f}".format(k_clf13.score(X_train, y_train)))
print("Test score: {:.3f}".format(k_clf13.score(X_test, y_test)))

### 이웃 = 15

In [ ]:
k_clf15 = KNeighborsClassifier(n_neighbors=15)
k_clf15.fit(X_train, y_train)
k_clf15.predict(X_test)
print("Train score: {:.3f}".format(k_clf15.score(X_train, y_train)))
print("Test score: {:.3f}".format(k_clf15.score(X_test, y_test)))

### 이웃 = 20

In [ ]:
k_clf20 = KNeighborsClassifier(n_neighbors=20)
k_clf20.fit(X_train, y_train)
k_clf20.predict(X_test)
print("Train score: {:.3f}".format(k_clf20.score(X_train, y_train)))
print("Test score: {:.3f}".format(k_clf20.score(X_test, y_test)))

In [ ]:
print("KNN confusion matrix : \n",
      confusion_matrix(y_train, k_clf13.predict(X_train)), "\n")

print("KNN classification report : \n",
      classification_report(y_train, k_clf13.predict(X_train)))